In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError

In [52]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')


In [66]:
ratings.shape

(25000095, 6)

In [67]:
movies.shape

(62423, 3)

In [53]:
user_id_map = {uid: idx for idx, uid in enumerate(ratings['userId'].unique())}
movie_id_map = {mid: idx for idx, mid in enumerate(ratings['movieId'].unique())}

ratings['user_id_enc'] = ratings['userId'].map(user_id_map)
ratings['movie_id_enc'] = ratings['movieId'].map(movie_id_map)

n_users = ratings['user_id_enc'].nunique()
n_movies = ratings['movie_id_enc'].nunique()

In [54]:
train, test = train_test_split(ratings, test_size=0.25, random_state=42)

In [55]:

user_model = Sequential([
    Embedding(input_dim=n_users, output_dim=32, input_length=1),
    Flatten()
])


movie_model = Sequential([
    Embedding(input_dim=n_movies, output_dim=32, input_length=1),
    Flatten()
])


d:\test\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [56]:
user_input = Input(shape=(1,))
movie_input = Input(shape=(1,))

In [57]:

user_vec = user_model(user_input)
movie_vec = movie_model(movie_input)

concat = Concatenate()([user_vec, movie_vec])

In [58]:

dense_block = Sequential([
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])


In [63]:
output = dense_block(concat)
model = Model(inputs=[user_input,movie_input], outputs=output)

In [64]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=MeanSquaredError(),
    metrics=[MeanAbsoluteError()]
)


In [61]:
model.summary()

Model: "functional_31"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_46      │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ keras_tensor_137CL… │ (None, 32)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_16       │ (None, 32)        │  1,889,504 │ input_layer_46[1… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_6       │ (None, 64)        │          0 │ keras_tensor_137… │
│ (Concatenate)       │                   │            │ sequential_16[1]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_17       │ (None, 1)         │      6,273 │ concatenate_6[1]… │
│ (Sequential)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,895,777 (7.23 MB)

 Trainable params: 1,895,777 (7.23 MB)

 Non-trainable params: 0 (0.00 B)

In [65]:
history = model.fit(
    [train['user_id_enc'], train['movie_id_enc']],
    train['rating'],
    batch_size=8192,
    epochs=3,
    validation_data=([test['user_id_enc'], test['movie_id_enc']], test['rating'])
)


Epoch 1/3
2289/2289 ━━━━━━━━━━━━━━━━━━━━ 51s 22ms/step - loss: 0.9339 - mean_absolute_error: 0.7107 - val_loss: 0.7075 - val_mean_absolute_error: 0.6407
Epoch 2/3
2289/2289 ━━━━━━━━━━━━━━━━━━━━ 47s 20ms/step - loss: 0.6795 - mean_absolute_error: 0.6265 - val_loss: 0.6712 - val_mean_absolute_error: 0.6204
Epoch 3/3
2289/2289 ━━━━━━━━━━━━━━━━━━━━ 50s 21ms/step - loss: 0.6343 - mean_absolute_error: 0.6043 - val_loss: 0.6489 - val_mean_absolute_error: 0.6105


In [ ]:
model.save("your_trained_movielens_model.h5")
print("✅ Sequential model saved!")